In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import numpy as np

In [2]:
# Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCHS = 25

# Directories
train_dir = 'dataset/train'
val_dir = 'dataset/val'
test_dir = 'dataset/test'

In [3]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 1562 images belonging to 4 classes.
Found 389 images belonging to 4 classes.
Found 18 images belonging to 4 classes.


In [4]:
# Load MobileNetV2 model with pretrained weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adding custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freezing the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

# # Unfreeze some layers and fine-tune the model
# for layer in base_model.layers[:100]:
#     layer.trainable = False
# for layer in base_model.layers[100:]:
#     layer.trainable = True

# # Re-compile the model with a lower learning rate
# model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

# # Continue training
# history_fine = model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // BATCH_SIZE,
#     validation_data=val_generator,
#     validation_steps=val_generator.samples // BATCH_SIZE,
#     epochs=EPOCHS // 2
# )


Epoch 1/25


C:\Users\jayesh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


48/48 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.6423 - loss: 2.4126 - val_accuracy: 0.8047 - val_loss: 0.5532
Epoch 2/25
 1/48 ━━━━━━━━━━━━━━━━━━━━ 23s 490ms/step - accuracy: 0.9062 - loss: 0.2352

C:\Users\jayesh\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9062 - loss: 0.2352 - val_accuracy: 0.8000 - val_loss: 0.5013
Epoch 3/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.8853 - loss: 0.3299 - val_accuracy: 0.8333 - val_loss: 0.3898
Epoch 4/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 0.0336 - val_accuracy: 1.0000 - val_loss: 0.0680
Epoch 5/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.8855 - loss: 0.2865 - val_accuracy: 0.8932 - val_loss: 0.2708
Epoch 6/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9062 - loss: 0.2799 - val_accuracy: 1.0000 - val_loss: 0.1475
Epoch 7/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.9301 - loss: 0.1861 - val_accuracy: 0.8828 - val_loss: 0.2825
Epoch 8/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8750 - loss: 0.3379 - val_accuracy: 0.8000 - val_loss: 0.2056
Epoch 9/25
48/48 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.9338 - loss: 0.1665 - val_accuracy: 0.9167 - val_loss: 0.2182
Epo

In [5]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('cotton_disease_model.h5')

# Load the model
model = tf.keras.models.load_model('cotton_disease_model.h5')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8889 - loss: 0.2110


Test accuracy: 0.8888888955116272


In [8]:
def predict_image(image_path):
    img = image.load_img(image_path, target_size=IMAGE_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction, axis=1)[0]
    class_labels = ['diseased cotton leaf', 'diseased cotton plant', 'fresh cotton leaf', 'fresh cotton plant']
    return class_labels[class_idx]

# Example prediction

image_path = 'dataset/test/fresh cotton leaf/d (396).jpg'
predicted_class = predict_image(image_path)
print(f'The predicted class is: {predicted_class}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
The predicted class is: fresh cotton leaf
